In [2]:
from datasets import load_dataset
import pinecone

dataset = load_dataset("squad")

Downloading: 5.27kB [00:00, 2.60MB/s]                   
Downloading: 2.36kB [00:00, 2.36MB/s]                   


Downloading: 30.3MB [00:00, 76.1MB/s]
Downloading: 4.85MB [00:00, 80.9MB/s]                   
100%|██████████| 2/2 [00:00<00:00, 499.98it/s]


Dataset squad downloaded and prepared to C:\Users\danhi\.cache\huggingface\datasets\squad\plain_text\1.0.0\d6ec3ceb99ca480ce37cdd35555d6cb2511d223b9150cce08a837ef62ffea453. Subsequent calls will reuse this data.


100%|██████████| 2/2 [00:00<00:00, 143.13it/s]


In [10]:
dataset['train'][-1]

{'id': '5735d259012e2f140011a0a1',
 'title': 'Kathmandu',
 'context': "Kathmandu Metropolitan City (KMC), in order to promote international relations has established an International Relations Secretariat (IRC). KMC's first international relationship was established in 1975 with the city of Eugene, Oregon, United States. This activity has been further enhanced by establishing formal relationships with 8 other cities: Motsumoto City of Japan, Rochester of the USA, Yangon (formerly Rangoon) of Myanmar, Xi'an of the People's Republic of China, Minsk of Belarus, and Pyongyang of the Democratic Republic of Korea. KMC's constant endeavor is to enhance its interaction with SAARC countries, other International agencies and many other major cities of the world to achieve better urban management and developmental programs for Kathmandu.",
 'question': 'What is KMC an initialism of?',
 'answers': {'text': ['Kathmandu Metropolitan City'], 'answer_start': [0]}}

In [11]:
from sentence_transformers import SentenceTransformer
embedder = SentenceTransformer('bert-base-nli-mean-tokens')

dataset_example = dataset['train'].map(
    lambda x: {
        'vector': embedder.encode(x['context']).tolist()
    }, batched=True, batch_size=16)

100%|██████████| 5475/5475 [06:58<00:00, 13.09ba/s]


In [29]:
dataset_example[0]

{'answers': {'text': ['Saint Bernadette Soubirous'], 'answer_start': [515]},
 'context': 'Architecturally, the school has a Catholic character. Atop the Main Building\'s gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.',
 'id': '5733be284776f41900661182',
 'question': 'To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?',
 'title': 'University_of_Notre_Dame',
 'vector': [-0.15370948612689972,
  1.141108512878

In [33]:
key = 'bed3cc56-6922-4ae9-ae26-775fb0197f69'
pinecone.init(api_key=key, environment='us-west1-gcp')

In [34]:
pinecone.create_index(name='squad-test', metric='euclidean', dimension=768)

In [28]:
index = pinecone.Index('squad-test')

In [36]:
dataset_example[0:10].keys()

dict_keys(['answers', 'context', 'id', 'question', 'title', 'vector'])

In [43]:
from tqdm.auto import tqdm  # for progress bar

batch_size = 100
for i in tqdm(range(0, len(dataset_example), batch_size)):
    # set end of current batch
    i_end = i + batch_size
    if i_end > len(dataset_example):
        # correct if batch is beyond dataset size
        i_end = len(dataset)
    batch = dataset_example[i: i_end]
    # upsert the batch
    index.upsert(vectors=zip(batch['id'], batch['vector'], batch['title']))